# Finding Self Avoiding Walks in an (m,n) grid from (0, 0) to (m,n):

In [1]:
import numpy as np
import time
from joblib import Parallel, delayed
import pickle

In [2]:
%%time

def take_step(current_position, allowed_moves, marker_with_border):
    # Returns the matrix indexs of the allowed next steps 
   
    possiblepos = current_position + allowed_moves
    # Calculates the 4 possible positions the walker could move to next
    
    available_bool = np.array([marker_with_border[row[1]][row[0]] == 0  for row in possiblepos])
    # Checks if each of the possible positions are empty (cells have 100 in them if part of the previous path or edge)
    # and creates an array (faster than list in testing) of booleans if it is available or not.
    
    return possiblepos[available_bool] # Returns the viable next steps

def main(m,n):
    # Returns the path of a self avoiding random walk from (0, 0) to (m, n) as a 1d array
    
    sizex = m + 1
    sizey = n +1
    # The size of the array has to account for the zeroth x/y rows
    
    top_pos = np.array([sizex, sizey])
    # Creates the array of the final position the walk is aiming to get to  

    path = np.zeros((sizex*sizey + 1, 2))
    # Makes an array for the maximum length of a walk i.e. a walk that visits all of the available coordinates
    # + 1 so that there is always at least 1 row of [-1, -1] at the end to index an cut off - kind of janky
    # See path_cut_off in Path Analyser.
    
    current_position = np.array([1,1])
    # Starting position is (1, 1) not (0, 0) to account for the edge
    
    path[0] = current_position
    # Sets the first position in the path to (1, 1) 
    
    allowed_moves = np.array(([1,0],[0,1],[-1,0],[0,-1]), dtype=int)
    # The 4 directions that a random walker can go to from its current position
    
    marker = np.zeros((sizey, sizex),dtype=int)
    # Makes an array on which the walker walks on. Cells will be set to 100 to keep track of where the 
    # walker has previously been.

    marker_with_border = np.pad(marker, 1,'constant',constant_values=100)
    # Creates an edge of 100 around the original marker array
    # This is to stop the take_step trying to index a matrix position outside of the array 
    # The initial current_position is set to (1, 1) to account for this shift and 1 is subtrated
    # from all of the values in the path at the end to counteract this 
    
    marker_with_border[current_position[1]][current_position[0]] = 100
    # Sets the initial position to 100 to stop the walker going back to it
    
    current_step = 1
    # To keep track of the next place in the path array to put a value (i.e. the first zero - useful to keep in mind for backtrack)
    
    while np.array_equal(current_position, top_pos) == False:
        # Keeps going until it reaches (m, n)
        
        candidates = take_step(current_position, allowed_moves, marker_with_border)
        # Calls take_step to get back an array of the valid next moves
        
        if len(candidates) == 0:
        # Checks if the walker is stuck
        
                while len(candidates) == 0:
                # Until the walker has a valid move
                
                    current_step -= 1 
                    # Goes back in the path array index by 1 i.e. the last non-zero value
                    # This is the matrix coordinates the walker currently is at
                    
                    path[current_step] = np.array([0,0])
                    # Erases the current position of the walker from the path - just in case the new path that leads to (m, n) is shorter than the amount of steps currently taken 
                    
                    current_position = path[current_step-1].astype(int)
                    # The new current_position of the walker needs to be the position before the walker got stuck
                    # As stated above, current_step points to the first zero so this is located 1 index before the 
                    # current step
                    
                    candidates = take_step(current_position, allowed_moves, marker_with_border)
                    # Calls take_step to get back an array of the valid next moves - to see if there are any now
    
        random_step = np.random.randint(0,len(candidates))
        # Makes a random index out of the available candidates array
        
        current_position = candidates[random_step]
        # Indexes the candidates array to get a random next move

        path[current_step] = current_position
        # Adds the new position of the walker to the path 
        
        marker_with_border[current_position[1]][current_position[0]] = 100
        # Marks that the walker has been to its current position
        
        current_step +=1        
        
    path -=1
    # Subtracts 1 from all of the elements in the path to account for the marker edge
    
    path = path.astype(int)
    # Makes sure all the elements are ints - for data reasons
    
    return np.ndarray.flatten(path) # Flattens path to 1d so that it can be compared to find uniques

Wall time: 0 ns


# For Singular: - To call the main function once

In [28]:
%%time
####### Change to specify dimensions of matrix
m = 9
n= 3
#######

####### Change to specify amount of simulations
amount_of_runs = 4000000
#######

paths = Parallel(n_jobs=12)(delayed(main)(m,n) for x in range(amount_of_runs))
# Parallel splits up the calls of main to different cores until a list of length amount_of_runs has been made
# n_jobs= for the amount of cpu core to utilise -1 for all available  

Wall time: 7min 39s


In [29]:
fileObj = open(str(m)+','+str(n)+'.txt', 'rb')
# Opens the serialised file for reading for the particular matrix

old_uniques = pickle.load(fileObj)
# Loads the first item in the serialised file - unique paths

total_simulations = pickle.load(fileObj)
# Loads the second item in the serialised file - total amount of simulations

paths = np.vstack((paths, old_uniques))
# Adds the new paths generated to the list of known unique paths so that the amount of unqiue paths  

total_simulations = total_simulations + amount_of_runs
# Updates the total amount of simulations run

fileObj.close()

fileObj = open(str(m)+','+str(n)+'.txt', 'wb')  
# Opens the serialised file for writing for the particular matrix

unique_rows = np.unique(paths, axis=0)
# Finds the new amount of unique paths

pickle.dump(unique_rows,fileObj)
# Writes all of the unique paths to the serialised file

print(total_simulations)
pickle.dump(total_simulations,fileObj)
# Writes the total amount of simulations to the serialised file


fileObj.close()

875000000


## For NEW Files - If there isn't a file already set up yet 

print ('New File')
total_simulations = amount_of_runs

fileObj = open(str(m)+','+str(n)+'.txt', 'wb')  
unique_rows = np.unique(paths, axis=0)


pickle.dump(unique_rows,fileObj)
print(total_simulations)
pickle.dump(total_simulations,fileObj)

fileObj.close()

# For Multiple:

In [3]:
def main_job(m,n):
    # Just the code for the single run but as a function so that it can be called multiple times
    
    amount_of_runs = 5*10**6
    paths = Parallel(n_jobs=12)(delayed(main)(m,n) for x in range(amount_of_runs))
    
    
    print('Opening')
    fileObj = open(str(m)+','+str(n)+'.txt', 'rb')
    
    old_uniques = pickle.load(fileObj)
    total_simulations = pickle.load(fileObj)
    paths = np.vstack((paths, old_uniques))
    
    total_simulations = total_simulations + amount_of_runs
    fileObj.close()

    fileObj = open(str(m)+','+str(n)+'.txt', 'wb')  

    unique_rows = np.unique(paths, axis=0)


    pickle.dump(unique_rows,fileObj)
    print(total_simulations)
    pickle.dump(total_simulations,fileObj)

    fileObj.close()
    print('Closing')

In [4]:
%%time
listofmat = np.array(([9,3],[9,3],[9,3],[9,3],[9,3],[9,3]))
# Specify which arrays and in what order the program does

for i in listofmat:
    main_job(i[0],i[1])

Opening
1330000000
Closing
Opening
1335000000
Closing
Opening
1340000000
Closing


KeyboardInterrupt: 